In [62]:
import pandas as pd
from sqlalchemy import create_engine

In [63]:
database = "extenso_customer_profile"
username = "root"
password = "root"
host = "localhost"
port = "3306"

In [64]:
db_uri = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}"


In [65]:
engine = create_engine(db_uri)

In [66]:
tables = pd.read_sql_table("product_category_map", engine)


In [67]:
from vanna.remote import VannaDefault

In [83]:
api_key = "1cdce4ff500d4205b6045ab5f2169a46"
vanna_model_name = "customer_profile1"

In [84]:
vn = VannaDefault(model = vanna_model_name,api_key = api_key)


In [85]:
vn.connect_to_mysql(host = host,dbname = database,user=username,password=password,port=3306)


In [86]:
schemas = vn.run_sql("select * from information_schema.columns where TABLE_SCHEMA = 'extenso_customer_profile'")

In [87]:
schemas

,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,COLUMN_NAME,ORDINAL_POSITION,COLUMN_DEFAULT,IS_NULLABLE,DATA_TYPE,CHARACTER_MAXIMUM_LENGTH,CHARACTER_OCTET_LENGTH,...,DATETIME_PRECISION,CHARACTER_SET_NAME,COLLATION_NAME,COLUMN_TYPE,COLUMN_KEY,EXTRA,PRIVILEGES,COLUMN_COMMENT,GENERATION_EXPRESSION,SRS_ID
0,def,extenso_customer_profile,product_category_map,module_id,1,None,YES,bigint,NaN,NaN,...,None,None,None,bigint,,,"select,insert,update,references",,,None
1,def,extenso_customer_profile,product_category_map,product_id,2,None,YES,bigint,NaN,NaN,...,None,None,None,bigint,,,"select,insert,update,references",,,None
2,def,extenso_customer_profile,product_category_map,product_type_id,3,None,YES,bigint,NaN,NaN,...,None,None,None,bigint,,,"select,insert,update,references",,,None
3,def,extenso_customer_profile,product_category_map,product_name,4,None,YES,varchar,765.0,3060.0,...,None,utf8mb4,utf8mb4_0900_ai_ci,varchar(765),,,"select,insert,update,references",,,None
4,def,extenso_customer_profile,product_category_map,product_category_id,5,None,YES,bigint,NaN,NaN,...,None,None,None,bigint,,,"select,insert,update,references",,,None
5,def,extenso_customer_profile,product_category_map,txn_flow,6,None,YES,varchar,300.0,1200.0,...,None,utf8mb4,utf8mb4_0900_ai_ci,varchar(300),,,"select,insert,update,references",,,None
6,def,extenso_customer_profile,rw_transaction_data,txn_id,1,None,NO,bigint,NaN,NaN,...,None,None,None,bigint,PRI,,"select,insert,update,references",,,None
7,def,extenso_customer_profile,rw_transaction_data,last_modified_date,2,None,NO,date,NaN,NaN,...,None,None,None,date,PRI,,"select,insert,update,references",,,None
8,def,extenso_customer_profile,rw_transaction_data,last_modified_date_bs,3,None,YES,varchar,30.0,120.0,...,None,utf8mb4,utf8mb4_0900_ai_ci,varchar(30),MUL,,"select,insert,update,references",,,None
9,def,extenso_customer_profile,rw_transaction_data,created_date,4,None,YES,date,NaN,NaN,...,None,None,None,date,MUL,,"select,insert,update,references",,,None


In [88]:
plan = vn.get_training_plan_generic(schemas)
plan

Train on Information Schema: def.extenso_customer_profile product_category_map
Train on Information Schema: def.extenso_customer_profile rw_transaction_data

In [89]:
vn.train(plan = plan)

In [90]:
vn.train(documentation='''Here is a table called rw_transaction_data which contains details of each transaction performed by users. The key aspects of this table are as follows:
a. Each row represents a transaction performed by a user.
b. Transactions are associated with specific products, which can include top-up services, airlines, fund transfers, and others.
c. Each transaction has a payer_account_id and a receiver_account_id.
d. Products are uniquely identified using a combination of module_id, product_id, and product_type_id.
e. Transaction details such as transaction amount, cashback amount, reward points, and revenue amount are provided.
f. The sample table includes transactions over a period of time for certain users.''')

Adding documentation....


'1104399-doc'

In [91]:
vn.train(documentation='''
Here is a table called Product Category Map Table which provides the mapping of products to their parent categories. The key aspects of this table are as follows:
a. It provides the mapping for each product to its parent category.
b. The product_category_id in this table corresponds to the ID in the product category table.
c. The txn_flow column indicates the transaction flow type, which can be:
i. Inflow: For products that credit a user account (e.g., another user’s account is credited).
ii. Outflow: For products that debit a user account (e.g., another user’s account is debited).
iii. Value Chain: For transactions that generate value for the client (e.g., top-up services, airline transactions).
''')

Adding documentation....


'1104400-doc'

In [92]:
vn.train(documentation=''' for question for each account id give me inflow,outflow and valuechain count
SELECT payer_account_id as account_id,
       SUM(CASE WHEN txn_flow = 'Inflow' THEN 1 ELSE 0 END) AS inflow_count,
       SUM(CASE WHEN txn_flow = 'Outflow' THEN 1 ELSE 0 END) AS outflow_count,
       SUM(CASE WHEN txn_flow = 'Value Chain' THEN 1 ELSE 0 END) AS valuechain_count
FROM rw_transaction_data
JOIN product_category_map ON rw_transaction_data.module_id = product_category_map.module_id
                         AND rw_transaction_data.product_id = product_category_map.product_id
                         AND rw_transaction_data.product_type_id = product_category_map.product_type_id
GROUP BY payer_account_id;
''')

Adding documentation....


'1104401-doc'

In [93]:
# vn.train(documentation = '''dont use customer profile table for calculation only use it as a reference. For all calculation use rw_transacation_data and product_category_map''')

In [96]:
from vanna.flask import VannaFlaskApp
VannaFlaskApp(vn,  allow_llm_to_see_data=True).run()

Your app is running at:
http://localhost:8084
 * Serving Flask app 'vanna.flask'
 * Debug mode: on
None
None
None
